In [ ]:
import os
from dotenv import load_dotenv
import pandas as pd
from lib.utils import enzyme_split30_preprocessing, read_h5, apply_embedding, read_fasta, calculate_f1, bootstrap_statistic, round_to_significance

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, f1_score, matthews_corrcoef
import joblib

In [ ]:
mode = "train"

In [ ]:
load_dotenv()
if mode == "train":
    path_to_non_enzymes = os.getenv("FASTA_NON_ENZYMES", "fasta not found")
    path_to_enzyme_csv = os.getenv("CSV30_ENZYMES", "csv not found")
    path_to_esm2_ne = os.getenv("ESM2_NON_ENZYMES", "esm2 ne not found")
    path_to_esm2 = os.getenv("ESM2_ENZYMES_SPLIT_30", "esm2 not found")
elif mode == "test":
    path_to_non_enzymes = os.getenv("TEST_FASTA_NON_ENZYMES", "fasta not found")
    path_to_enzyme_csv = os.getenv("TEST_CSV_ENZYMES", "csv not found")
    path_to_esm2_ne = os.getenv("TEST_ESM2_NON_ENZYMES", "esm2 ne not found")
    path_to_esm2 = os.getenv("TEST_ESM2_ENZYMES", "esm2 not found")

#path_to_enzyme_csv_esm2 = os.getenv("CSV30_ENZYMES_ESM25_APPLIED")
#path_to_non_enzymes_esm2 = os.getenv("NON_ENZYMES_ESM2_APPLIED")

In [ ]:
enzymes=pd.read_csv(path_to_enzyme_csv, delimiter=",")
print("RAW ENZYMES SIZE: " + str(enzymes.count()))
enzymes = enzyme_split30_preprocessing(enzymes)
print("PREPROCESSED ENZYMES SIZE: " + str(enzymes.count()))
enzymes.head()

In [ ]:
enzymes = apply_embedding(read_h5(path_to_esm2, False), enzymes)
enzymes.head()

In [45]:
non_enzymes = read_fasta(path_to_non_enzymes)
print("NON-ENZYMES SIZE: " + str(non_enzymes.count()))
non_enzymes.head()

NON-ENZYMES SIZE: ID          12593
Sequence    12593
dtype: int64


,ID,Sequence
0,Q5E769,MRKVLKKAALCTFGFSMLFGCASEEDTIVMAPVPVVQNQFEPTTEW...
1,O66960,MRYFVIGTMIALAGLLVGGGVGSYFTSSKLLKQFQNIPGSPIVLSA...
2,G5EFD5,MICASKITMLGLLVMCTLGGVLGKVDIRQTTANKAFMETMRADGYE...
3,Q5UQQ8,MIICNFMNICMIDPNDVYKWFMEFSLDSYDWVMINNVYSMGLFADG...
4,P0C1T8,DCEQHTDCSAASGPVYCCQDSDCCGGVDYICTNYGQCVRHF


In [44]:
non_enzymes = apply_embedding(read_h5(path_to_esm2_ne, False), non_enzymes)
non_enzymes.head()

,ID,Embedding,Embedding_x,Embedding_y,Sequence,Label
0,A0A023PXC2,"[0.026780527, -0.07408093, 0.01210466, -0.0393...","[0.026780527, -0.07408093, 0.01210466, -0.0393...",0.026781,MLPLCLTFLSFFLSLGGSFKAVMTKEEADGTTEAAACLFWIFNWTV...,0
1,A0A023PXC7,"[0.070358254, -0.016387884, 0.009544486, -0.00...","[0.070358254, -0.016387884, 0.009544486, -0.00...",0.070358,MAPPTLITANCCCETEVKYFKYCSTSLFVLILFNSWITVDLVAEKP...,0
2,A0A023PXD3,"[-0.013247637, -0.0578524, 0.015567223, -0.030...","[-0.013247637, -0.0578524, 0.015567223, -0.030...",-0.013248,MTRLPPIPRMTVTLTTRPAVPTCNEGSSILHYIYIPIYEPNEQKEK...,0
3,A0A023PXE5,"[0.025439993, -0.044117454, 0.053229067, -0.03...","[0.025439993, -0.044117454, 0.053229067, -0.03...",0.025440,MDLYPPASWAALVPFCKALTFKVPVVLGNRNPSPPSPLPPMALSLS...,0
4,A0A023PXG3,"[0.014469501, -0.045669124, 0.011890223, -0.02...","[0.014469501, -0.045669124, 0.011890223, -0.02...",0.014470,MATENNKNPAIRFLLSVVGSGNSLSILNGLFLSFKTILASSSATLL...,0


In [49]:
non_enzymes.head()
"""
import numpy as np
# Split data
enzymes["Label"] = 1
non_enzymes["Label"] = 0

bin = pd.concat([enzymes[["Label", "Embedding"]], non_enzymes[["Label", "Embedding"]]], ignore_index=True)

bin = bin.sample(frac=1, random_state=42).reset_index(drop=True)

X = [value for value in bin["Embedding"]]
y = bin["Label"]
if mode == "train":
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    y_test = np.array(y_test)
elif mode == "test":
    X_test = X
    y_test = y
    y_test = np.array(y_test)

"""

'\nimport numpy as np\n# Split data\nenzymes["Label"] = 1\nnon_enzymes["Label"] = 0\n\nbin = pd.concat([enzymes[["Label", "Embedding"]], non_enzymes[["Label", "Embedding"]]], ignore_index=True)\n\nbin = bin.sample(frac=1, random_state=42).reset_index(drop=True)\n\nX = [value for value in bin["Embedding"]]\ny = bin["Label"]\nif mode == "train":\n    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)\n    y_test = np.array(y_test)\nelif mode == "test":\n    X_test = X\n    y_test = y\n    y_test = np.array(y_test)\n    \n'

In [38]:
bin

,Label,Embedding
0,0,0.064773
1,1,"[-0.07571557, -0.05513923, 0.040098827, -0.048..."
2,0,0.00066
3,1,"[0.010099938, -0.04706414, -0.04221819, -0.030..."
4,0,0.11424
...,...,...
21158,0,-0.048186
21159,0,0.073527
21160,1,"[-0.029177392, 0.004335931, -0.050398204, 0.03..."
21161,1,"[0.040678505, -0.04778647, -0.023326838, -0.00..."


In [31]:
if mode == "train":
    k = 7
    knn_classifier = KNeighborsClassifier(n_neighbors=k)

    # Fit the classifier to the training data
    knn_classifier.fit(X_train, y_train)

    # Make predictions on the test data
    y_pred = knn_classifier.predict(X_test) # y_pred are predicted labels for embeddings at index
elif mode == "test":
    knn_classifier = joblib.load('./var/level0_esm2_knn.pkl')
    y_pred = knn_classifier.predict(X_test) # y_pred are predicted labels for embeddings at index


C:\Users\smith\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\_array_api.py:380: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  array = numpy.asarray(array, order=order, dtype=dtype)


ValueError: Expected 2D array, got 1D array instead:
array=[list([0.0055709546, -0.014277849, -0.03570309, -0.08091941, -0.02612455, -0.066844374, -0.052445084, 0.10058481, 0.0012422202, -0.00301929, -0.027380409, 0.019839233, -0.03931493, 0.043500513, -0.028747438, -0.021962225, -0.027961573, 0.0071349805, -0.076295085, -0.10173023, -0.00029900565, -0.03398687, 0.052100543, 0.030976672, -0.0125312805, -0.04341234, 0.058127273, -0.04557044, -0.06402024, -0.026778007, 0.03427238, 0.015160872, -0.06343482, 0.031650834, 0.016723955, 0.052891377, 0.023838244, 0.07036089, -0.0803682, 0.021424066, -0.012166171, 0.03802609, 0.0434203, 0.09217236, -0.04924293, -0.045087393, 0.05743799, -0.036735248, -0.026302405, 0.0046659354, -0.034732636, 0.011262455, 0.011676586, -0.017367069, -0.0018704287, 0.06903515, 0.010457859, 0.03546396, -0.0007537877, 0.04120836, 0.024685638, 0.0054714656, -0.088947505, 0.01771194, -0.032729197, -0.008549258, -0.033653058, -0.02245706, 0.13369578, -0.00686292, 0.111

In [ ]:
with open(f'./var/{mode}_knn_esm2_y_pred.txt', 'w') as file:
    for label in y_pred:
        file.write(f"{label}\n")
with open(f'./var/{mode}_knn_esm2_y_test.txt', 'w') as file:
    for label in y_test:
        file.write(f"{label}\n")

In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
cm = confusion_matrix(y_test, y_pred)
# Define custom colors (e.g., green and purple)
colors = ['yellow', 'purple']

# Create a colormap using custom colors
cmap = mcolors.ListedColormap(colors)

# Plot the confusion matrix heatmap
plt.imshow(cm, interpolation='nearest', cmap=cmap)
plt.colorbar()
plt.xticks([0, 1], ["Predicted 0", "Predicted 1"])
plt.yticks([0, 1], ["Actual 0", "Actual 1"])

plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')

for i in range(2):
    for j in range(2):
        plt.text(j, i, str(cm[i, j]), ha='center', va='center', color='white', fontsize=16)

plt.show()

In [ ]:
initial_f1 = calculate_f1(np.array(y_test), y_pred)
mean_f1, se_f1, ci_95 = bootstrap_statistic(y_test, y_pred, calculate_f1)
rounded_mean_f1, rounded_se_f1 = round_to_significance(mean_f1, se_f1)

In [ ]:
print(f"ESM2 KNN:")
print(f"  - Accuracy: {accuracy_score(y_test, y_pred)}")
print(f"  - Initial F1 Score: {initial_f1:.2f}")
print(f"  - MCC: {matthews_corrcoef(y_test, y_pred)}")
print(f"  - Mean F1 ± SE F1: {rounded_mean_f1} ± {rounded_se_f1}")
print(f"  - 95% CI: [{ci_95[0]:.2f}, {ci_95[1]:.2f}]")

In [ ]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
# roc curve
fpr, tpr, thresholds = roc_curve(y_test, y_pred)

# Calculate the AUC (Area Under the ROC Curve)
roc_auc = auc(fpr, tpr)
# Plot the ROC curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve for KNN (k=7)')
plt.legend(loc='lower right')
plt.show()

In [ ]:
if mode == "train":
    joblib.dump(knn_classifier, './var/level0_esm2_knn.pkl')